# Test if packages were installed correctly

## Basic packages

In [ ]:
import os
import sqlite3
from math import ceil


## Pandas, Matplotlib, Seaborn, Scikit-Learn

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# for better resolution plots
%config InlineBackend.figure_format = 'retina' 

# Setting seaborn style
sns.set()

In [ ]:
from matplotlib import __version__ as mplver
print(mplver)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [ ]:
from sklearn import datasets

In [ ]:
dataset = datasets.load_iris()
df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
df.head()


In [ ]:
plt.scatter(df.iloc[:,0],df.iloc[:,1])
plt.show()

In [ ]:
sns.scatterplot(df, x='petal length (cm)', y='petal width (cm)')

## Test ProfileReport

In [ ]:
from ydata_profiling import ProfileReport

In [ ]:
profile = ProfileReport(
    df, 
    title='Iris Data',
    correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": False},
        "kendall": {"calculate": False},
        "phi_k": {"calculate": False},
        "cramers": {"calculate": False},
    },
)

In [ ]:
profile.to_notebook_iframe()


## Test Packages for SOM

In [ ]:
from IPython.display import YouTubeVideo
import minisom

In [ ]:
import sompy
from sompy.visualization.mapview import View2D
from sompy.visualization.bmuhits import BmuHitsView
from sompy.visualization.hitmap import HitMapView

In [ ]:
# This som implementation does not have a random seed parameter
# We're going to set it up ourselves
np.random.seed(42)

sm = sompy.SOMFactory().build(
    df.values, 
    mapsize=[8, 16],  # NEEDS TO BE A LIST
    initialization='random', 
    neighborhood='gaussian',
    training='batch',
    lattice='hexa',
    component_names=dataset.feature_names
)
sm.train(n_job=4, verbose='info', train_rough_len=50, train_finetune_len=50)

In [ ]:
plt.rcParams['figure.dpi'] = 48

sns.set()
view2D = View2D(8, 16, "", text_size=10)
view2D.show(sm, col_sz=2, what='codebook')

plt.show()

In [ ]:
u = sompy.umatrix.UMatrixView(9, 9, 'umatrix', show_axis=True, text_size=8, show_text=True)

UMAT = u.show(
    sm, 
    distance=3,
)

In [ ]:
vhts  = BmuHitsView(12,12,"Hits Map")
vhts.show(sm, anotate=True, onlyzeros=False, labelsize=12, cmap="Blues")
plt.show()

## Test Graphviz

In [ ]:
import graphviz

In [ ]:
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split


In [ ]:
kmclust = KMeans(n_clusters=3, init='random', n_init=10, random_state=1)
km_labels = kmclust.fit_predict(df)

In [ ]:
# Preparing the data
X = df
y = km_labels

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Fitting the decision tree
dt = DecisionTreeClassifier(random_state=42, max_depth=3)
dt.fit(X_train, y_train)


In [ ]:
# Visualizing the decision tree
dot_data = export_graphviz(dt, out_file=None, 
                           feature_names=X.columns.to_list(),
                           filled=True,
                           rounded=True,
                           special_characters=True)  
graphviz.Source(dot_data)